In [ ]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import asyncio

In [10]:
SEASONS = list(range(2018, 2025))

In [11]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR)

In [ ]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=False,
            args=['--disable-blink-features=AutomationControlled']
        )
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            viewport={'width': 1920, 'height': 1080}
        )
        page = await context.new_page()
        
        try:
            for i in range(1, retries + 1):
                if i > 1:
                    await asyncio.sleep(sleep * i)
                
                try:
                    await page.goto(url, wait_until='networkidle', timeout=60000)
                    print(await page.title())
                    
                    # Wait for selector with longer timeout
                    await page.wait_for_selector(selector, timeout=60000)
                    
                    # Add a small delay to ensure content is rendered
                    await asyncio.sleep(2)
                    
                    html = await page.inner_html(selector)
                    break
                    
                except PlaywrightTimeout:
                    print(f"Timeout error on {url} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                except Exception as e:
                    print(f"Error on {url}: {e} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                        
        finally:
            await browser.close()
    
    return html

In [15]:
season = 2019
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [16]:
url

'https://www.basketball-reference.com/leagues/NBA_2019_games.html'

In [ ]:
html = await get_html(url, "#content .filter")

Just a moment...
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
Just a moment...
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
Just a moment...
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
Just a moment...
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
